In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import sys
import os
import torch
import time
import random
import numpy as np
import wandb
from tqdm import tqdm

from models.resnet import resnet18, resnet34, resnet50

wandb.login(key="ef091b9abcea3186341ddf8995d62bde62d7469e")
wandb.init(project="PBL-2", name="resnet50-cutmix")


"""
# 학습 재현성 고정
def fix_seed(seed, deterministic=False):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
    else:
        torch.backends.cudnn.benchmark = True  # 성능 향상을 위해 True로 변경

deterministic=True와 benchmark=False는 확실히 학습 속도를 저하시킬 수 있습니다.
특히 torch.backends.cudnn.benchmark=False는 CUDA가 최적의 알고리즘을 찾기 위한 
벤치마킹을 수행하지 않게 만들어 성능이 떨어질 수 있습니다.
# 속도 우선 설정 -> 완벽한 재현성은 보장되지 않음 

fix_seed(2025, deterministic=False)
"""

# WandB 설정
config = {
    "model": "resnet50",
    "batch_size": 128,
    "num_epochs": 100,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "seed": 2025,
    "deterministic": False,
}
wandb.config.update(config)

# CIFAR-100 데이터셋 로드
transform_train = transforms.Compose([
    transforms.RandomHorizonalFlip()
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=config["batch_size"], shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=config["batch_size"], shuffle=False, num_workers=16)


def cutmix(batch, alpha):
    data, targets = batch

    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)

    image_h, image_w = data.shape[2:]
    cx = np.random.uniform(0, image_w)
    cy = np.random.uniform(0, image_h)
    w = image_w * np.sqrt(1 - lam)
    h = image_h * np.sqrt(1 - lam)
    x0 = int(np.round(max(cx - w / 2, 0)))
    x1 = int(np.round(min(cx + w / 2, image_w)))
    y0 = int(np.round(max(cy - h / 2, 0)))
    y1 = int(np.round(min(cy + h / 2, image_h)))

    data[:, :, y0:y1, x0:x1] = shuffled_data[:, :, y0:y1, x0:x1]
    targets = (targets, shuffled_targets, lam)

    return data, targets


class CutMixCollator:
    def __init__(self, alpha):
        self.alpha = alpha

    def __call__(self, batch):
        batch = torch.utils.data.dataloader.default_collate(batch)
        batch = cutmix(batch, self.alpha)
        return batch


class CutMixCriterion:
    def __init__(self, reduction):
        self.criterion = nn.CrossEntropyLoss(reduction=reduction)

    def __call__(self, preds, targets):
        targets1, targets2, lam = targets
        return lam * self.criterion(
            preds, targets1) + (1 - lam) * self.criterion(preds, targets2)




def train(model, trainloader, criterion, optimizer, device, epoch):
    """
    학습 함수 - 0.5 확률로 CutMix 적용
    """
    model.train()   # 모델을 학습 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 0.5 확률로 CutMix 적용
        r = np.random.rand(1)
        if r < 0.5:  # 50% 확률로 CutMix 적용
            # CutMix 적용
            indices = torch.randperm(inputs.size(0)).to(device)
            shuffled_labels = labels[indices]
            
            # 혼합 비율 결정 (Beta(1,1) = 균등분포)
            lam = np.random.beta(1.0, 1.0)
            
            # 랜덤 박스 좌표 생성
            image_h, image_w = inputs.shape[2:]
            cx = np.random.uniform(0, image_w)
            cy = np.random.uniform(0, image_h)
            w = image_w * np.sqrt(1 - lam)
            h = image_h * np.sqrt(1 - lam)
            x0 = int(np.round(max(cx - w / 2, 0)))
            x1 = int(np.round(min(cx + w / 2, image_w)))
            y0 = int(np.round(max(cy - h / 2, 0)))
            y1 = int(np.round(min(cy + h / 2, image_h)))
            
            # 이미지 섞기
            inputs[:, :, y0:y1, x0:x1] = inputs[indices, :, y0:y1, x0:x1]
            
            # 실제 혼합 비율 계산 (영역 기반)
            lam = 1 - ((y1 - y0) * (x1 - x0) / (image_h * image_w))
            
            # 모델 순전파
            outputs = model(inputs)
            
            # CutMix 손실 계산
            loss = lam * criterion(outputs, labels) + (1 - lam) * criterion(outputs, shuffled_labels)
        else:
            # 일반 순전파 및 손실 계산
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # 정확도 계산 (CutMix를 적용했을 때는 원래 레이블로 평가)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if (i + 1) % 50 == 0:  # 50 배치마다 출력
            print(f'Epoch [{epoch+1}], Batch [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
    
    epoch_loss = running_loss / len(trainloader)
    accuracy = 100.0 * correct / total
    
    train_time = time.time() - start_time
    
    # 학습 세트에 대한 성능 출력
    print(f'Train set: Epoch: {epoch+1}, Average loss:{epoch_loss:.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f} '
          f'Top-1 Accuracy: {accuracy:.4f}%, Time consumed:{train_time:.2f}s')
    
    return epoch_loss, accuracy

def evaluate(model, testloader, criterion, device, epoch):
    """
    평가 함수
    """
    model.eval()  # 모델을 평가 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    
    test_loss = 0.0
    correct = 0
    total = 0
    
    # 그래디언트 계산 비활성화
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 순전파
            outputs = model(inputs)
            
            # 손실 계산
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            # 정확도 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # 평균 손실 및 정확도 계산
    test_loss = test_loss / len(testloader)
    accuracy = 100.0 * correct / total
    
    # 평가 시간 계산
    eval_time = time.time() - start_time
    
    # 테스트 세트에 대한 성능 출력 (document_7 스타일)
    print(f'Test set: Epoch: {epoch+1}, Average loss:{test_loss:.4f}, '
          f'Top-1 Accuracy: {accuracy:.4f}%, Time consumed:{eval_time:.2f}s')
    print()
    
    return test_loss, accuracy

# 메인 학습 루프
def main_training_loop(model, trainloader, testloader, criterion, optimizer, device, num_epochs=10):
    """
    메인 학습 루프
    """
    best_acc = 0.0
    
    # tqdm을 사용한 진행 상황 표시
    for epoch in tqdm(range(num_epochs)):
        # 학습
        train_loss, train_acc = train(model, trainloader, criterion, optimizer, device, epoch)
        
        # 평가
        test_loss, test_acc = evaluate(model, testloader, criterion, device, epoch)
        
        # WandB에 로깅
        wandb.log({
            "epoch": epoch + 1,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "test_loss": test_loss,
            "test_accuracy": test_acc
        })
            
        # 최고 정확도 모델 저장
        if test_acc > best_acc:
            best_acc = test_acc
            print(f'new best accuracy: {best_acc:.2f}%')
            
            # 모델 저장
            model_path = f'best_model_{wandb.run.name}.pth'
            torch.save(model.state_dict(), model_path)
            
            # WandB에 모델 아티팩트 저장
            wandb.save(model_path)
    
    print(f'finish! best accuracy: {best_acc:.2f}%')
    wandb.run.summary["best_accuracy"] = best_acc

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 초기화
model = resnet50().to(device)  
criterion = nn.CrossEntropyLoss()  # 손실 함수 정의
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])  # 옵티마이저 정의

# WandB에 모델 구조 기록
wandb.watch(model, log="all")

# GPU 가속
if torch.cuda.device_count() > 1:
    print(f"{torch.cuda.device_count()}개의 GPU를 사용합니다.")
    model = nn.DataParallel(model)

# 훈련 시작 시간 기록
start_time = time.time()

# 메인 학습 루프 호출
main_training_loop(
    model=model,
    trainloader=trainloader,
    testloader=testloader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=config["num_epochs"]
)

# 훈련 종료 시간 기록 및 출력
end_time = time.time()
total_time = end_time - start_time
wandb.log({"total_training_time": total_time})

print(f"Total training time: {total_time:.2f} seconds")

# WandB 실행 종료
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Files already downloaded and verified
Files already downloaded and verified
Using device: cuda
2개의 GPU를 사용합니다.


  0%|                                                                                                        | 0/20 [00:00<?, ?it/s]

Epoch [1], Batch [50/391], Loss: 4.0298
Epoch [1], Batch [100/391], Loss: 3.9639
Epoch [1], Batch [150/391], Loss: 3.7676
Epoch [1], Batch [200/391], Loss: 3.7366
Epoch [1], Batch [250/391], Loss: 3.1741
Epoch [1], Batch [300/391], Loss: 3.3485
Epoch [1], Batch [350/391], Loss: 3.0976
Train set: Epoch: 1, Average loss:3.6294, LR: 0.001000 Top-1 Accuracy: 14.2620%, Time consumed:136.77s


  5%|████▊                                                                                          | 1/20 [02:40<50:56, 160.89s/it]

...

 95%|█████████████████████████████████████████████████████████████████████████████████████████▎    | 19/20 [50:26<02:41, 161.63s/it]

Test set: Epoch: 19, Average loss:3.3095, Top-1 Accuracy: 50.7000%, Time consumed:23.21s

Epoch [20], Batch [50/391], Loss: 0.0545
Epoch [20], Batch [100/391], Loss: 0.0499
Epoch [20], Batch [150/391], Loss: 0.0306
Epoch [20], Batch [200/391], Loss: 0.0350
Epoch [20], Batch [250/391], Loss: 0.0907
Epoch [20], Batch [300/391], Loss: 0.0412
Epoch [20], Batch [350/391], Loss: 0.1017
Train set: Epoch: 20, Average loss:0.0736, LR: 0.001000 Top-1 Accuracy: 97.6180%, Time consumed:131.23s


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [53:09<00:00, 159.49s/it]

Test set: Epoch: 20, Average loss:2.9964, Top-1 Accuracy: 53.3200%, Time consumed:32.37s

finish! best accuracy: 54.13%
Total training time: 3189.73 seconds


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▄▆▇███████▇██████▇█
test_loss,█▄▂▁▁▁▁▂▃▃▄▅▅▅▅▅▆▆█▆
total_training_time,▁
train_accuracy,▁▂▃▄▅▅▆▇▇███████████
train_loss,█▆▅▄▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁
best_accuracy,54.13
epoch,20
learning_rate,0.001
test_accuracy,53.32
